In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

In [3]:
#Open csv file
data = pd.read_csv(r"/home/kchen/Documents/leak/Other/Data/hep/hep.csv", index_col="CASEID", na_values = -99, low_memory=False)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/kchen/Documents/leak/Other/Data/hep/hep.csv'

In [ ]:
data.shape

(25595, 373)

In [ ]:
features = ['PRNCPTX', 'CPT', 'CONCPT1', 'OTHERCPT1', 'OPERYR', 'SEX', 'AGE', 'HEIGHT', 'WEIGHT', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'TRANST','WNDCLAS','ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT',  'PRALBUM', 'PRBILI', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'SEPSHOCKPATOS', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'DOPERTOD','OTHBLEED']

In [ ]:
hep_features = [col for col in data if col.startswith('HEP')]

In [ ]:
outcomes = ['HEP_PEAKCREAT_UNK','HEP_DRAINS_30D','HEP_DRAINS_UNK','HEP_NEOADJ_UNK', 'HEP_DRAINS_BILI_UNK','HEP_CON_OP_ABLATION_UNK', 'HEP_RECON_UNK','HEP_DRAINS_BILI','HEP_DRAIN_BILI_UNK', 'DDRAINSREMOVAL','HEP_DRAINS_REMOVAL_UNK','HEP_INVASIVE','HEP_INVASIVE_UNK','HEP_INVASIVE_TYPE','HEP_PEAKINR','HEP_PEAKINR_UNK','HEP_PEAKBILI','HEP_PEAKBILI_UNK','HEP_LIVERFAIL','HEP_LIVERFAIL_GRADE','HEP_PATHRES','HEP_HISTOLOGIC','HEP_TSTAGE','HEP_NSTAGE','HEP_MSTAGE','HEP_SEC_HISTOLOGIC','HEP_SEC_NUMTUMORS','HEP_SEC_TUMORSIZE','HEP_BENIGN_HISTOLOGIC','HEP_BENIGN_LESION']

In [ ]:
hep_features = list(set(hep_features) - set(outcomes))

In [ ]:
hep_features

['HEP_NEOADJ',
 'HEP_RECON',
 'HEP_CON_OP_ABLATION',
 'HEP_LIVERTEXT',
 'HEP_PEAKCREAT',
 'HEP_PRINGLE',
 'HEP_CON_ABLATION_140101',
 'HEP_BILIARYSTENT',
 'HEP_LAPTHOR',
 'HEP_APPROACH',
 'HEP_NEOTHERAPY_140101',
 'HEP_DRAINS',
 'HEP_VIRAL',
 'HEP_BILELEAKAGE',
 'HEP_CON_PARTRES']

In [ ]:
#Remove unwanted columns
data = data[features + hep_features]

In [ ]:
#Create BMI column
lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

data['BMI'] = data['WEIGHT']/(data['HEIGHT']**2)
print(min(data['BMI']))
print(max(data['BMI']))

10.958882654126503
89.27363187762685


In [ ]:
print(data.columns.tolist())

['PRNCPTX', 'CPT', 'CONCPT1', 'OTHERCPT1', 'OPERYR', 'SEX', 'AGE', 'HEIGHT', 'WEIGHT', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'TRANST', 'WNDCLAS', 'ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'SEPSHOCKPATOS', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME', 'OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'DOPERTOD', 'OTHBLEED', 'HEP_NEOADJ', 'HEP_RECON', 'HEP_CON_OP_ABLATION', 'HEP_LIVERTEXT', 'HEP_PEAKCREAT', 'HEP_PRINGLE', 'HEP_CON_ABLATION_140101', 'HEP_BILIARYSTENT', 'HEP_LAPTHOR', 'HEP_APPROACH', 'HEP_NEOT

In [ ]:
data.shape

(25595, 88)

In [ ]:
data['HEP_BILELEAKAGE'].value_counts(dropna=False)

No                                                           23538
Yes-persistent drainage, drain continued on or after POD3      621
Yes-clinical diagnosis, percutaneous drainage performed        484
Yes-persistent drainage, percutaneous drainage performed       290
Yes-clinical diagnosis, drain continued on or after POD3       278
Unknown                                                        192
Yes-clinical diagnosis, reoperation performed                  111
Yes-persistent drainage, reoperation performed                  52
Yes-clinical diagnosis, spontaneous wound drainage              29
Name: HEP_BILELEAKAGE, dtype: int64

In [ ]:
#Remove unwanted indication from COL_INDICATION
data.drop(data[data['HEP_BILELEAKAGE'] == 'Unknown'].index, inplace=True)
data['HEP_BILELEAKAGE'].value_counts()

No                                                           23538
Yes-persistent drainage, drain continued on or after POD3      621
Yes-clinical diagnosis, percutaneous drainage performed        484
Yes-persistent drainage, percutaneous drainage performed       290
Yes-clinical diagnosis, drain continued on or after POD3       278
Yes-clinical diagnosis, reoperation performed                  111
Yes-persistent drainage, reoperation performed                  52
Yes-clinical diagnosis, spontaneous wound drainage              29
Name: HEP_BILELEAKAGE, dtype: int64

In [ ]:
data.loc[data['HEP_BILELEAKAGE'] == 'No', 'HEP_BILELEAKAGE'] = False
data.loc[data['HEP_BILELEAKAGE'] == 'Yes-persistent drainage, drain continued on or after POD3', 'HEP_BILELEAKAGE'] = False
data.loc[data['HEP_BILELEAKAGE'] == 'Yes-clinical diagnosis, drain continued on or after POD3', 'HEP_BILELEAKAGE'] = False
data.loc[data['HEP_BILELEAKAGE'] != False, 'HEP_BILELEAKAGE'] = True

data['HEP_BILELEAKAGE'].value_counts()

False    24437
True       966
Name: HEP_BILELEAKAGE, dtype: int64

In [ ]:
data['HEP_BILELEAKAGE'].value_counts(normalize=True)

False    0.961973
True     0.038027
Name: HEP_BILELEAKAGE, dtype: float64

In [ ]:
data.groupby('HEP_DRAINS')['HEP_BILELEAKAGE'].value_counts(normalize=True)

HEP_DRAINS  HEP_BILELEAKAGE
No          False              0.977790
            True               0.022210
Unknown     False              0.956522
            True               0.043478
Yes         False              0.942381
            True               0.057619
Name: HEP_BILELEAKAGE, dtype: float64

In [ ]:
#Cast ASA class as ordered categorical
cat_type = CategoricalDtype(categories=['None assigned','1-No Disturb','2-Mild Disturb','3-Severe Disturb','4-Life Threat','5-Moribund'], ordered=True)
data['ASACLAS'].astype(cat_type)


CASEID
4903256    3-Severe Disturb
4905354    3-Severe Disturb
4909128      2-Mild Disturb
4919340      2-Mild Disturb
4919479    3-Severe Disturb
                 ...       
7340721    3-Severe Disturb
7340725      2-Mild Disturb
7340756      2-Mild Disturb
7340837    3-Severe Disturb
7341071      2-Mild Disturb
Name: ASACLAS, Length: 25403, dtype: category
Categories (6, object): ['None assigned' < '1-No Disturb' < '2-Mild Disturb' < '3-Severe Disturb' < '4-Life Threat' < '5-Moribund']

In [ ]:
#Convert 90+ to 91 and AGE to int32
data.loc[data['AGE'] == '90+', 'AGE'] = 91
pd.to_numeric(data['AGE'], downcast='integer')

CASEID
4903256    52.0
4905354    78.0
4909128    35.0
4919340    47.0
4919479    74.0
           ... 
7340721    73.0
7340725    35.0
7340756    71.0
7340837    65.0
7341071    76.0
Name: AGE, Length: 25403, dtype: float64

In [ ]:
data['HEP_APPROACH'].value_counts()

Open (planned)                                  18616
Laparoscopic                                     3833
Laparoscopic w/ open assist                      1239
Laparoscopic w/ unplanned conversion to open      939
Robotic                                           527
Robotic w/ open assist                            125
Robotic w/ unplanned conversion to open            58
Hybrid                                             43
Unknown                                             8
Hybrid w/ open assist                               6
Other MIS approach                                  3
Other                                               3
Hybrid w/ unplanned conversion to open              1
Endoscopic w/ unplanned conversion to open          1
Other MIS approach w/ open assist                   1
Name: HEP_APPROACH, dtype: int64

In [ ]:
data.loc[data['HEP_APPROACH'] == 'Laparoscopic', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Laparoscopic w/ open assist', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Laparoscopic w/ unplanned conversion to open', 'HEP_APPROACH'] = 'Laparoscopic w/ conversion'

data.loc[data['HEP_APPROACH'] == 'Robotic', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Robotic w/ open assist', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Robotic w/ unplanned conversion to open', 'HEP_APPROACH'] = 'Laparoscopic w/ conversion'

data.loc[data['HEP_APPROACH'] == 'NOTES', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'NOTES w/ open assist', 'HEP_APPROACH'] = 'Laparoscopic w/ open assist'
data.loc[data['HEP_APPROACH'] == 'NOTES w/ unplanned conversion to open', 'HEP_APPROACH'] = 'Laparoscopic w/ conversion'

data.loc[data['HEP_APPROACH'] == 'Other MIS approach', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Other MIS approach w/ open assist', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Other MIS approach w/ unplanned conversion to open', 'HEP_APPROACH'] = 'Laparoscopic w/ conversion'

data.loc[data['HEP_APPROACH'] == 'Hybrid', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Hybrid w/ open assist', 'HEP_APPROACH'] = 'Laparoscopic'
data.loc[data['HEP_APPROACH'] == 'Hybrid w/ unplanned conversion to open', 'HEP_APPROACH'] = 'Laparoscopic w/ conversion'

data.loc[data['HEP_APPROACH'] == 'SILS', 'HEP_APPROACH'] = 'Laparoscopic'

data.loc[data['HEP_APPROACH'] == 'Other', 'HEP_APPROACH'] = 'Unknown'

data.loc[data['HEP_APPROACH'] == 'Endoscopic w/ unplanned conversion to open', 'HEP_APPROACH'] = 'Laparoscopic w/ conversion'

data['HEP_APPROACH'].value_counts()

Open (planned)                18616
Laparoscopic                   5777
Laparoscopic w/ conversion      999
Unknown                          11
Name: HEP_APPROACH, dtype: int64

In [ ]:
data['HEP_NEOTHERAPY_140101'].value_counts(dropna=False)

NaN                                                                                                    17875
Preoperative systemic chemotherapy                                                                      6038
Portal vein embolization                                                                                 470
Preoperative systemic chemotherapy,Portal vein embolization                                              331
Other type                                                                                               210
Locoregional interarterial infusion                                                                      150
Preoperative systemic chemotherapy,Other type                                                             77
Locoregional liver ablation                                                                               73
Preoperative systemic chemotherapy,Locoregional liver ablation                                            56
Preoperative system

In [ ]:
data['CHEMO'] = 0
data['EMBO'] = 0
data['INFU'] = 0
data['ABLA'] = 0

In [ ]:
data['HEP_NEOTHERAPY_140101'] = data['HEP_NEOTHERAPY_140101'].fillna('unknown')

In [ ]:
data['HEP_NEOTHERAPY_140101'] = data['HEP_NEOTHERAPY_140101'].fillna('unknown')

In [ ]:
data.loc[data['HEP_NEOTHERAPY_140101'].str.contains('Portal vein embolization'), 'EMBO'] = 1
data['EMBO'].value_counts()

0    24526
1      877
Name: EMBO, dtype: int64

In [ ]:
data.loc[data['HEP_NEOTHERAPY_140101'].str.contains('Preoperative systemic chemotherapy'), 'CHEMO'] = 1
data['CHEMO'].value_counts()

0    18837
1     6566
Name: CHEMO, dtype: int64

In [ ]:
data.loc[data['HEP_NEOTHERAPY_140101'].str.contains('Locoregional interarterial infusion'), 'INFU'] = 1
data['INFU'].value_counts()

0    25181
1      222
Name: INFU, dtype: int64

In [ ]:
data.loc[data['HEP_NEOTHERAPY_140101'].str.contains('Locoregional liver ablation'), 'ABLA'] = 1
data['ABLA'].value_counts()

0    25234
1      169
Name: ABLA, dtype: int64

In [ ]:
data = data.drop(['HEP_NEOTHERAPY_140101'], axis=1)

In [ ]:
data.loc[data['HEP_CON_ABLATION_140101'] == 'RFA ablation,Microwave ablation', 'HEP_CON_ABLATION_140101'] = 'RFA ablation'
data.loc[data['HEP_CON_ABLATION_140101'] == 'Microwave ablation,Other ablation', 'HEP_CON_ABLATION_140101'] = 'Microwave ablation'

data.loc[data['HEP_CON_ABLATION_140101'] == 'RFA ablation,Other ablation', 'HEP_CON_ABLATION_140101'] = 'RFA ablation'
data.loc[data['HEP_CON_ABLATION_140101'] == 'RFA ablation,Microwave ablation', 'HEP_CON_ABLATION_140101'] = 'RFA ablation'
data.loc[data['HEP_CON_ABLATION_140101'] == 'RFA ablation,Alcohol ablation', 'HEP_CON_ABLATION_140101'] = 'RFA ablation'
data.loc[data['HEP_CON_ABLATION_140101'] == 'Microwave ablation,Alcohol ablation', 'HEP_CON_ABLATION_140101'] = 'Microwave ablation'
data.loc[data['HEP_CON_ABLATION_140101'] == 'RFA ablation,Cryoablation', 'HEP_CON_ABLATION_140101'] = 'Microwave ablation'


In [ ]:
data['HEP_CON_ABLATION_140101'].value_counts()

RFA ablation          1583
Microwave ablation    1443
Other ablation         259
Alcohol ablation        15
Cryoablation            13
Name: HEP_CON_ABLATION_140101, dtype: int64

In [ ]:
data.loc[data['HEP_CON_PARTRES'] == '3', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '4', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '5', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '6', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '7', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '8', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '9', 'HEP_CON_PARTRES'] = '3+'
data.loc[data['HEP_CON_PARTRES'] == '10 or more', 'HEP_CON_PARTRES'] = '3+'
data['HEP_CON_PARTRES'].value_counts()

0          12688
1           6822
2           3011
3+          2512
Unknown      370
Name: HEP_CON_PARTRES, dtype: int64

In [35]:
data.to_csv(r'/home/kchen/Documents/leak/Other/Data/hep/clean_hep.csv')

In [1]:
data = pd.read_csv(r'/home/kchen/Documents/leak/old_leak/Other/Data/hep/clean_hep.csv')

NameError: name 'pd' is not defined

In [ ]:
pd.DataFrame((data.isnull().sum()/len(data)).sort_values(ascending=False)).head(20)

,0
DOPERTOD,0.986261
PRPT,0.984254
HEP_LAPTHOR,0.948549
CONCPT1,0.921702
HEP_CON_ABLATION_140101,0.869582
HEP_PEAKCREAT,0.513916
PRPTT,0.368854
OTHERCPT1,0.280833
PRINR,0.136598
PRALBUM,0.121875


In [ ]:
drop = ['PRPT']
data.drop(drop, axis=1, inplace=True)

In [ ]:
drop = ['HEP_PEAKCREAT']
data.drop(drop, axis=1, inplace=True)

In [ ]:
drop = ['OUPNEUMO', 'CDARREST', 'CDMI', 'SUPINFEC', 'WNDINFD', 'ORGSPCSSI', 'URNINFEC', 'OTHDVT', 'PULEMBOL', 'RENAINSF', 'OPRENAFL', 'DEHIS', 'REINTUB', 'FAILWEAN', 'CNSCVA', 'OTHSYSEP', 'OTHBLEED', 'OTHSESHOCK', 'DOPERTOD']
data.drop(drop, axis=1, inplace=True)

In [ ]:
num_cols = []
for col in data.columns:
    if len(data[col].unique()) > 100:
        num_cols.append(col)

In [ ]:
num_cols = ['AGE','HEIGHT','WEIGHT', 'BMI','OPTIME','PRSODM','PRBUN','PRCREAT','PRALBUM','PRBILI','PRALKPH','PRWBC','PRHCT','PRPLATE','PRPTT','PRINR','OPTIME']

In [ ]:
cat_cols = list(set(data.columns) - set(num_cols) - set('COL_ANASTOMOTIC'))

In [ ]:
for col in cat_cols:
    data[col].fillna(value='Unknown', inplace=True)

In [ ]:
num_imputer = SimpleImputer(strategy = 'median')

# Imputation for numeric:
for ncol in num_cols:
    data[ncol] = num_imputer.fit_transform(data[ncol].values.reshape(-1, 1))

In [ ]:
for col in num_cols:
    data[col] = RobustScaler().fit_transform(data[col].values.reshape(-1,1))

In [ ]:
le=LabelEncoder()
for col in cat_cols:
        data[col] = le.fit_transform(data[col].astype(str))

In [ ]:
data.head()

,CASEID,PRNCPTX,CPT,CONCPT1,OTHERCPT1,OPERYR,SEX,AGE,HEIGHT,WEIGHT,...,HEP_APPROACH,HEP_DRAINS,HEP_VIRAL,HEP_BILELEAKAGE,HEP_CON_PARTRES,BMI,CHEMO,EMBO,INFU,ABLA
0,7646,1,2,317,473,2,1,52.0,1.4986,60.781328,...,2,2,3,0,0,27.064420,0,0,0,0
1,7653,0,0,317,473,2,2,78.0,1.7272,110.676448,...,0,0,3,0,1,37.099662,0,0,0,0
2,7665,0,0,317,266,2,1,35.0,1.6510,64.410064,...,2,2,3,1,3,23.629775,0,0,0,0
3,7705,0,0,317,291,2,2,47.0,1.7272,59.874144,...,2,2,3,0,1,20.070309,0,0,0,0
4,7706,0,0,317,473,2,2,74.0,1.6510,71.667536,...,2,0,2,0,1,26.292285,0,0,0,0


In [ ]:
data.to_csv('clean_hep.csv')

In [ ]:
hep_cols = []
for col in data.columns:
    if col.startswith('HEP'):
        hep_cols.append(col)


In [ ]:
hep_cols

['HEP_NEOADJ',
 'HEP_RECON',
 'HEP_CON_OP_ABLATION',
 'HEP_LIVERTEXT',
 'HEP_PEAKCREAT',
 'HEP_PRINGLE',
 'HEP_CON_ABLATION_140101',
 'HEP_BILIARYSTENT',
 'HEP_LAPTHOR',
 'HEP_APPROACH',
 'HEP_DRAINS',
 'HEP_VIRAL',
 'HEP_BILELEAKAGE',
 'HEP_CON_PARTRES']